<a href="https://colab.research.google.com/github/ankita2002/LLMS/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3: Transformers, T5, Machine Translation

This homework guides you through implementing a complete T5 (Text-To-Text Transfer Transformer) encoder-decoder architecture from scratch for machine translation. You will build the encoder stack, decoder stack with cross-attention, and train the model on English-German translation.

**Total Points: 20**

**Instructions:**
1. Complete all tasks in this notebook
2. Ensure your code runs without errors
3. Submit both this notebook and any additional files created
4. Write clear explanations for your approach
5. Train your model on the English-German translation dataset

## Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Optional, Tuple, List, Dict

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


## Dataset Preparation

We'll use the WMT English-German translation dataset. This dataset contains parallel sentences in English and German, perfect for training a sequence-to-sequence translation model.

In [2]:
!pip install -q datasets

In [4]:
import os
from collections import Counter
from datasets import load_dataset
import tqdm

print("Initializing WMT Dataset Loader (No backups, real data)...")

# 1. Load the Real WMT Dataset
# We use WMT16 (German-English), a standard benchmark.
# We take the first 50,000 examples to keep RAM usage efficient while ensuring variety.
# If you want the full dataset (4.5M pairs), remove the [:50000] slice.
dataset = load_dataset("wmt16", "de-en", split="train[:50000]")

print(f"Successfully loaded {len(dataset)} pairs from WMT16.")

# 2. Extract pairs into memory
translation_pairs = []
for item in dataset:
    # WMT data structure is usually {'translation': {'de': '...', 'en': '...'}}
    en_text = item['translation']['en']
    de_text = item['translation']['de']
    translation_pairs.append((en_text, de_text))

# 3. Build Vocabulary (Character-level)
# We stick to character-level to match your previous logic.
# Note: For SOTA results on WMT, you would usually use BPE (Byte Pair Encoding),
# but character-level is robust for learning deep learning fundamentals.
print("Building vocabulary from dataset...")

counter = Counter()
for en, de in translation_pairs:
    counter.update(en) # Keep case sensitivity? standard is usually to keep it for WMT
    counter.update(de)

# Filter rare characters to keep vocab clean (optional, but good for real web data)
MIN_FREQ = 5
common_chars = [char for char, count in counter.items() if count >= MIN_FREQ]

# Add special tokens
special_tokens = ['<pad>', '<sos>', '<eos>', '<unk>']
chars = special_tokens + sorted(common_chars)
vocab_size = len(chars)

print(f'Shared vocabulary size: {vocab_size}')

# 4. Mappings
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

PAD_IDX = stoi['<pad>']
SOS_IDX = stoi['<sos>']
EOS_IDX = stoi['<eos>']
UNK_IDX = stoi['<unk>']

# 5. Encoding/Decoding Functions
def encode(s):
    """Encode string to token IDs (Character level)."""
    # We do NOT lower() here to preserve information, unlike the toy example
    return [stoi.get(c, UNK_IDX) for c in s]

def decode(l):
    """Decode token IDs to string."""
    return ''.join([itos[i] for i in l if i != PAD_IDX])

print(f"Special tokens: PAD={PAD_IDX}, SOS={SOS_IDX}, EOS={EOS_IDX}, UNK={UNK_IDX}")

# 6. Train/Val Split (80/20)
n_train = int(0.8 * len(translation_pairs))
train_pairs = translation_pairs[:n_train]
val_pairs = translation_pairs[n_train:]

print("-" * 40)
print(f"Train pairs: {len(train_pairs)}")
print(f"Val pairs:   {len(val_pairs)}")
print("-" * 40)
print("Sample Data (Real WMT):")
print(f"  EN: {train_pairs[5][0]}") # Random index to show real sentences
print(f"  DE: {train_pairs[5][1]}")
print(f"  Encoded: {encode(train_pairs[5][0])}")

Initializing WMT Dataset Loader (No backups, real data)...
Successfully loaded 50000 pairs from WMT16.
Building vocabulary from dataset...
Shared vocabulary size: 113
Special tokens: PAD=0, SOS=1, EOS=2, UNK=3
----------------------------------------
Train pairs: 40000
Val pairs:   10000
----------------------------------------
Sample Data (Real WMT):
  EN: Please rise, then, for this minute' s silence.
  DE: Ich bitte Sie, sich zu einer Schweigeminute zu erheben.
  Encoded: [45, 69, 62, 58, 76, 62, 4, 75, 66, 76, 62, 13, 4, 77, 65, 62, 71, 13, 4, 63, 72, 75, 4, 77, 65, 66, 76, 4, 70, 66, 71, 78, 77, 62, 9, 4, 76, 4, 76, 66, 69, 62, 71, 60, 62, 15]


## TASK 1: T5 Encoder Implementation (6 points)

Build the encoder stack that processes source sequences with bidirectional attention. The encoder is similar to BERT but designed for the T5 architecture.

**1.1**: *Encoder Block* — Implement T5 encoder block with bidirectional self-attention, feed-forward network, layer normalization, and residual connections

**1.2**: *Encoder Stack* — Stack multiple encoder blocks with positional encoding and token embeddings

**1.3**: *Encoder Testing* — Test encoder implementation and verify bidirectional attention works correctly


In [5]:
# Helper function: Positional Encoding
#This code builds a fixed table of sine/cos position vectors, then adds the right row to each token embedding so the Transformer knows token order.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))

        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        return x + self.pe[:x.size(1), :].unsqueeze(0)

In [6]:
class FeedForward(nn.Module):
    def __init__(self, n_embd, hidden_size, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, hidden_size), #expands the vector
            nn.GELU(), # A smooth activation function, Adds non-linearity
            nn.Dropout(dropout), #Randomly zeros some values during training, helps reduce overfitting
            nn.Linear(hidden_size, n_embd), #reduces the expanded size back to original
            nn.Dropout(dropout),
        )
    def forward(self, x): return self.net(x)

In [7]:
# Helper function: Scaled Dot-Product Attention
def scaled_dot_product_attention(query, key, value, mask=None, dropout=None):
    """
    Compute scaled dot-product attention.

    Args:
        query: [batch_size, seq_len, d_k] or [batch_size, n_heads, seq_len, d_k]
        key: [batch_size, seq_len, d_k] or [batch_size, n_heads, seq_len, d_k]
        value: [batch_size, seq_len, d_v] or [batch_size, n_heads, seq_len, d_v]
        mask: [batch_size, seq_len, seq_len] or None (0s for masked positions)
        dropout: Dropout layer or None
    Returns:
        output: [batch_size, seq_len, d_v] or [batch_size, n_heads, seq_len, d_v]
        attention_weights: [batch_size, seq_len, seq_len] or [batch_size, n_heads, seq_len, seq_len]
    """
    d_k = query.size(-1)

    # Compute attention scores: Q @ K^T / sqrt(d_k) using dot product
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    #it id divided by sqrt(d_k) to keep numbers stable
    #Without this scaling, scores can get too large and softmax becomes unstable.

    # Apply mask if provided (set masked positions to -inf before softmax)
    # Mask is used to block attention to some positions.
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)

    # Apply softmax to get attention weights
    attention_weights = F.softmax(scores, dim=-1)

    # Apply dropout if provided
    if dropout is not None:
        attention_weights = dropout(attention_weights)

    # Multiply attention weights by values
    output = torch.matmul(attention_weights, value)

    return output, attention_weights

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_embd, n_head, dropout=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        self.head_dim = n_embd // n_head
        self.scale = self.head_dim ** -0.5
        self.query = nn.Linear(n_embd, n_embd)
        self.key = nn.Linear(n_embd, n_embd)
        self.value = nn.Linear(n_embd, n_embd)
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        B, T, C = q.shape
        q = self.query(q).view(B, -1, self.n_head, self.head_dim).transpose(1, 2)
        k = self.key(k).view(B, -1, self.n_head, self.head_dim).transpose(1, 2)
        v = self.value(v).view(B, -1, self.n_head, self.head_dim).transpose(1, 2)
        scores = (q @ k.transpose(-2, -1)) * self.scale
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        attn = self.dropout(F.softmax(scores, dim=-1))
        out = (attn @ v).transpose(1, 2).contiguous().view(B, -1, C)
        return self.proj(out), attn

### Task 1.1 Implement T5 encoder block

Implement a T5 encoder block with:
- Bidirectional multi-head self-attention (all positions can attend to all positions)
- Feed-forward network
- Layer normalization (pre-norm architecture: normalize before sub-layers)
- Residual connections around each sub-layer

The encoder block should follow this structure:
1. Self-attention: `x = x + dropout(attention(layer_norm(x)))`
2. Feed-forward: `x = x + dropout(ffn(layer_norm(x)))`

In [9]:
class T5EncoderBlock(nn.Module):
    """T5 Encoder Block with bidirectional self-attention."""
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        ## YOUR_CODE_STARTS_HERE
        # TODO: Initialize:

        # - Multi-head self-attention (bidirectional, no masking)
        self.self_attn = MultiHeadAttention(n_embd=d_model, n_head=n_heads, dropout=dropout)
        # - Feed-forward network
        self.ffn = FeedForward(n_embd=d_model, hidden_size=d_ff, dropout=dropout)
        # - Two layer normalization layers (one for attention, one for FFN)
        self.ln_attn = nn.LayerNorm(d_model)
        self.ln_ffn = nn.LayerNorm(d_model)
        # - Dropout layer
        self.dropout = nn.Dropout(dropout)

        ## YOUR_CODE_ENDS_HERE

    def forward(self, x, mask=None):
        ## YOUR_CODE_STARTS_HERE
        # TODO: Implement encoder block with residual connections (pre-norm)
        # 1. Self-attention: x = x + dropout(attention(layer_norm(x)))
        x_norm = self.ln_attn(x) #pre-norm: normalize before attention
        attn_out, _ = self.self_attn(x_norm, x_norm, x_norm, mask=mask) #(Q,K,V,mask) self attention
        # mask is optional. In encoder you usually use it to ignore <pad> tokens.
        x = x + self.dropout(attn_out)

        # 2. Feed-forward: x = x + dropout(ffn(layer_norm(x)))
        # Note: Apply layer norm BEFORE the sub-layer (pre-norm architecture)
        #FFN improves each token’s representation on its own.
        #It’s a small neural net: expand -> nonlinearity -> shrink back.
        x_norm = self.ln_ffn(x)
        ffn_out = self.ffn(x_norm)
        x = x + self.dropout(ffn_out)
        ## YOUR_CODE_ENDS_HERE

        return x

#normalize -> self-attention -> add back (residual) -> normalize -> feed-forward -> add back (residual)

In [10]:
# Test your implementation
encoder_block = T5EncoderBlock(d_model=512, n_heads=8, d_ff=2048)
x = torch.randn(2, 10, 512)
output = encoder_block(x)
print(f"Encoder block output shape: {output.shape}")
## RESULT_CHECKING_POINT -> torch.Size([2, 10, 512])

Encoder block output shape: torch.Size([2, 10, 512])


### Task 1.2 Build encoder stack


In [11]:
class T5Encoder(nn.Module):
    """T5 Encoder: Stack of encoder blocks with embeddings and positional encoding."""
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, max_len=5000, dropout=0.1):
        super().__init__()
        ## YOUR_CODE_STARTS_HERE
        # TODO: Initialize:
        # - Token embedding layer (ID-> vector)
        self.token_emb = nn.Embedding(vocab_size, d_model)
        # - Positional encoding (add order info)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)
        # Dropout after embeddings + positional encoding
        self.dropout = nn.Dropout(dropout)
        # - Stack of encoder blocks (n_layers)
        self.blocks = nn.ModuleList([
            T5EncoderBlock(d_model=d_model, n_heads=n_heads, d_ff=d_ff, dropout=dropout)
            for _ in range(n_layers)
        ])
        # - Final layer normalization (optional, but common in T5)
        self.ln_f = nn.LayerNorm(d_model)
        ## YOUR_CODE_ENDS_HERE

    def forward(self, x, mask=None):
        ## YOUR_CODE_STARTS_HERE
        # TODO: Implement encoder forward pass
        # 1. Token embeddings
        x = self.token_emb(x)  # (B, T) -> (B, T, d_model)
        # 2. Add positional encoding
        x = self.pos_enc(x)    # (B, T, d_model)
        # 3. Apply dropout
        x = self.dropout(x)
        # 4. Pass through encoder blocks
        for block in self.blocks:
            x = block(x, mask=mask)
        # 5. Apply final layer norm
        x = self.ln_f(x)
        ## YOUR_CODE_ENDS_HERE

        return x

# Test your implementation
encoder = T5Encoder(vocab_size=1000, d_model=512, n_heads=8, n_layers=6, d_ff=2048)
x = torch.randint(0, 1000, (2, 10))
output = encoder(x)
print(f"Encoder output shape: {output.shape}")
## RESULT_CHECKING_POINT -> torch.Size([2, 10, 512])

Encoder output shape: torch.Size([2, 10, 512])


### Task 1.3 Test encoder implementation

In [12]:
def test_encoder(encoder, sample_text, max_len=50):
    """Test encoder and visualize attention patterns."""
    encoder.eval()

    # Encode text → ensure tensors go to the correct device
    encoded = torch.tensor(encode(sample_text[:max_len]), device=device).unsqueeze(0)

    # Forward pass
    with torch.no_grad():
        output = encoder(encoded)

    print(f"Input text: {sample_text[:max_len]}")
    print(f"Input shape: {encoded.shape}")
    print(f"Encoder output shape: {output.shape}")
    print(f"Encoder output mean: {output.mean().item():.4f}")
    print(f"Encoder output std: {output.std().item():.4f}")

    return output

## TASK 2: T5 Decoder with Cross-Attention (7 points)

Build the decoder stack with causal self-attention and cross-attention to encoder outputs. The decoder uses masked self-attention (causal) for autoregressive generation and cross-attention to attend to the encoder's output.

**2.1**: *Cross-Attention Mechanism* — Implement cross-attention where queries come from decoder states, and keys/values come from encoder outputs

**2.2**: *T5 Decoder Block* — Implement decoder block with masked self-attention, cross-attention, and feed-forward network

**2.3**: *Decoder Stack* — Stack multiple decoder blocks with positional encoding and token embeddings

### Task 2.1 Implement cross-attention mechanism

In [13]:
class CrossAttentionHead(nn.Module):
    """Single head of cross-attention (decoder attends to encoder)."""
    def __init__(self, head_size, n_embd, dropout):
        super().__init__()
        ## YOUR_CODE_STARTS_HERE
        # Query comes from decoder, Key and Value come from encoder
        self.q = nn.Linear(n_embd, head_size, bias=False)
        self.k = nn.Linear(n_embd, head_size, bias=False)
        self.v = nn.Linear(n_embd, head_size, bias=False)

        self.scale = head_size ** -0.5
        self.dropout = nn.Dropout(dropout)
        ## YOUR_CODE_ENDS_HERE

    def forward(self, decoder_states, encoder_states):
        ## YOUR_CODE_STARTS_HERE
        # decoder_states: (B, T_dec, n_embd)
        # encoder_states: (B, T_enc, n_embd)
        Q = self.q(decoder_states)
        K = self.k(encoder_states)
        V = self.v(encoder_states)

        # scores: (B, T_dec, T_enc)
        scores = (Q @ K.transpose(-2, -1)) * self.scale

        attn = F.softmax(scores, dim=-1)     # (B, T_dec, T_enc)
        attn = self.dropout(attn)

        out = attn @ V                       # (B, T_dec, head_size)
        ## YOUR_CODE_ENDS_HERE
        return out

In [14]:
class CrossMultiHeadAttention(nn.Module):
    """Multi-head cross-attention for T5."""
    def __init__(self, n_head, head_size, n_embd, dropout):
        super().__init__()
        ## YOUR_CODE_STARTS_HERE
        self.heads = nn.ModuleList([
            CrossAttentionHead(head_size=head_size, n_embd=n_embd, dropout=dropout)
            for _ in range(n_head)
        ])
        self.proj = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        ## YOUR_CODE_ENDS_HERE

    def forward(self, decoder_states, encoder_states):
        ## YOUR_CODE_STARTS_HERE
        out = torch.cat([h(decoder_states, encoder_states) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))   # (B, T_dec, n_embd)
        ## YOUR_CODE_ENDS_HERE
        return out

In [15]:
# Test your implementation
cross_attn = CrossMultiHeadAttention(n_head=8, head_size=64, n_embd=512, dropout=0.1)
decoder_states = torch.randn(2, 10, 512)  # (batch, decoder_seq_len, d_model)
encoder_states = torch.randn(2, 15, 512)  # (batch, encoder_seq_len, d_model)
output = cross_attn(decoder_states, encoder_states)
print(f"Cross-attention output shape: {output.shape}")
## RESULT_CHECKING_POINT -> torch.Size([2, 10, 512])

Cross-attention output shape: torch.Size([2, 10, 512])


### Task 2.2 Implement T5 decoder block

Implement the decoder block with:
- Masked self-attention (causal, for autoregressive generation)
- Cross-attention (decoder attends to encoder)
- Feed-forward network
- Three layer normalization layers (pre-norm)
- Residual connections

In [16]:
def generate_causal_mask(seq_len):
    return torch.tril(torch.ones(seq_len, seq_len))

In [19]:
class T5DecoderBlock(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        head_size = n_embd // n_head
        ## YOUR_CODE_STARTS_HERE
        # Masked self-attention
        self.self_attn = MultiHeadAttention(n_embd=n_embd, n_head=n_head, dropout=dropout)
        #Cross-attention (decoder -> encoder)
        self.cross_attn = CrossMultiHeadAttention(
            n_head=n_head, head_size=head_size, n_embd=n_embd, dropout=dropout
        )
        #Feed-forward
        self.ffn = FeedForward(n_embd=n_embd, hidden_size=4 * n_embd, dropout=dropout)

        # Pre-norm LayerNorms (one before each sub-layer)
        self.ln_self = nn.LayerNorm(n_embd)
        self.ln_cross = nn.LayerNorm(n_embd)
        self.ln_ffn = nn.LayerNorm(n_embd)

        # Dropout around each sub-layer output
        self.dropout = nn.Dropout(dropout)
        # Causal mask: allow attending to past + self only
        causal = torch.tril(torch.ones(block_size, block_size))
        self.register_buffer("causal_mask", causal)

        ## YOUR_CODE_ENDS_HERE

    def forward(self, x, encoder_output):
        ## YOUR_CODE_STARTS_HERE
        B, T, C = x.shape
        # mask self attention
        mask = self.causal_mask[:T, :T].unsqueeze(0).unsqueeze(0)  # (1,1,T,T)
        x_norm = self.ln_self(x)
        self_out, _ = self.self_attn(x_norm, x_norm, x_norm, mask=mask)
        x = x + self.dropout(self_out)

        #cross-attention
        x_norm = self.ln_cross(x)
        cross_out = self.cross_attn(x_norm, encoder_output)
        x = x + self.dropout(cross_out)

        #Feed-Forward
        x_norm = self.ln_ffn(x)
        ffn_out = self.ffn(x_norm)
        x = x + self.dropout(ffn_out)

        ## YOUR_CODE_ENDS_HERE
        return x

### Task 2.3 Build decoder stack


In [27]:
class T5Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, block_size, max_len, dropout):
        super().__init__()
        ## YOUR_CODE_STARTS_HERE
        # Token embedding + positional encoding
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_enc = PositionalEncoding(d_model, max_len=max_len)
        self.dropout = nn.Dropout(dropout)

        # Stack of decoder blocks
        self.blocks = nn.ModuleList([
            T5DecoderBlock(n_embd=d_model, n_head=n_heads, block_size=block_size, dropout=dropout)
            for _ in range(n_layers)
        ])

        # Final layer norm
        self.ln_f = nn.LayerNorm(d_model)
        ## YOUR_CODE_ENDS_HERE

    def forward(self, x, encoder_output):
        ## YOUR_CODE_STARTS_HERE
        # x: (B, T_dec) token ids
        # encoder_output: (B, T_enc, d_model)

        #Token embeddings
        x = self.token_emb(x)          # (B, T_dec, d_model)
        #Positional encoding
        x = self.pos_enc(x)            # (B, T_dec, d_model)
        #Dropout
        x = self.dropout(x)

        #Decoder blocks (masked self-attn + cross-attn + FFN)
        for block in self.blocks:
            x = block(x, encoder_output)

        #Final layer norm
        x = self.ln_f(x)

        return x
        ## YOUR_CODE_ENDS_HERE

In [28]:
print("Testing T5DecoderBlock...")
decoder_block = T5DecoderBlock(n_embd=512, n_head=8, block_size=128, dropout=0.1)
x = torch.randn(2, 10, 512)
encoder_output = torch.randn(2, 15, 512)
output = decoder_block(x, encoder_output)
print(f"Decoder block output shape: {output.shape}")

Testing T5DecoderBlock...
Decoder block output shape: torch.Size([2, 10, 512])


## TASK 3: Full T5 Model and Machine Translation (7 points)

Combine encoder and decoder into a complete T5 model and train it on English-German machine translation.

**3.1**: *Complete T5 Model* — Combine encoder and decoder stacks with shared token embeddings and language modeling head

**3.2**: *Translation Dataset Preparation* — Prepare English-German translation dataset with proper batching and padding

**3.3**: *Training and Testing* — Train model on translation task and evaluate performance

### Task 3.1 Build complete T5 model


In [29]:
class T5Model(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, block_size, max_len=5000, dropout=0.1):
        super().__init__()
        ## YOUR_CODE_STARTS_HERE
        self.shared_emb = nn.Embedding(vocab_size, d_model)

        # Encoder + Decoder stacks
        self.encoder = T5Encoder(
            vocab_size=vocab_size,
            d_model=d_model,
            n_heads=n_heads,
            n_layers=n_layers,
            d_ff=d_ff,
            max_len=max_len,
            dropout=dropout,
        )
        self.decoder = T5Decoder(
            vocab_size=vocab_size,
            d_model=d_model,
            n_heads=n_heads,
            n_layers=n_layers,
            d_ff=d_ff,
            block_size=block_size,
            max_len=max_len,
            dropout=dropout,
        )

        # Replace their embeddings with the shared one
        self.encoder.token_emb = self.shared_emb
        self.decoder.token_emb = self.shared_emb

        # Language modeling head (decoder states -> vocab logits)
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

        # Tie lm_head weights to the shared embedding (common trick)
        self.lm_head.weight = self.shared_emb.weight

        ## YOUR_CODE_ENDS_HERE

    def forward(self, src, tgt):
        ## YOUR_CODE_STARTS_HERE
        pad_idx = globals().get("PAD_IDX", None)
        if pad_idx is not None:
            # (B, 1, 1, T_src) mask for encoder self-attn (1 = keep, 0 = pad)
            src_mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
        else:
            src_mask = None

        encoder_output = self.encoder(src, mask=src_mask)          # (B, T_src, d_model)
        decoder_output = self.decoder(tgt, encoder_output)         # (B, T_tgt, d_model)
        logits = self.lm_head(decoder_output)                      # (B, T_tgt, vocab_size)

        ## YOUR_CODE_ENDS_HERE
        return logits

### Task 3.2 Prepare translation dataset


In [30]:
def get_translation_batch(split, batch_size=4):
    """Get a batch of English-German translation pairs."""
    pairs = train_pairs if split == 'train' else val_pairs

    # Random sampling
    indices = torch.randint(0, len(pairs), (batch_size,))

    src_batch = []
    tgt_batch = []

    for idx in indices:
        en_text, de_text = pairs[idx]
        src_ids = encode(en_text)
        # German: Add SOS at start, EOS at end
        tgt_ids = [SOS_IDX] + encode(de_text) + [EOS_IDX]

        src_batch.append(torch.tensor(src_ids, dtype=torch.long))
        tgt_batch.append(torch.tensor(tgt_ids, dtype=torch.long))

    # Pad
    src_batch = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    tgt_batch = torch.nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_IDX)

    # Teacher forcing split
    tgt_input = tgt_batch[:, :-1] # Input: SOS ... Token
    tgt_output = tgt_batch[:, 1:] # Target: Token ... EOS

    return src_batch.to(device), tgt_input.to(device), tgt_output.to(device)

In [31]:
BLOCK_SIZE = 512 # Increased from 64 to 512 to handle sentences > 64 chars
BATCH_SIZE = 16

model = T5Model(
    vocab_size=vocab_size,
    d_model=128,
    n_heads=4,
    n_layers=2,
    d_ff=512,
    block_size=BLOCK_SIZE
).to(device)

print(f"Model initialized with block_size={BLOCK_SIZE}")

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Updated Batch Function with Truncation
def get_safe_batch(split, batch_size=4, max_len=BLOCK_SIZE):
    src_batch, tgt_in, tgt_out = get_translation_batch(split, batch_size)

    # Truncate if they exceed block_size
    if src_batch.size(1) > max_len:
        src_batch = src_batch[:, :max_len]
    if tgt_in.size(1) > max_len:
        tgt_in = tgt_in[:, :max_len]
        tgt_out = tgt_out[:, :max_len] # Output must match input length

    return src_batch, tgt_in, tgt_out

print("Starting training...")
model.train()

Model initialized with block_size=512
Starting training...


T5Model(
  (shared_emb): Embedding(113, 128)
  (encoder): T5Encoder(
    (token_emb): Embedding(113, 128)
    (pos_enc): PositionalEncoding()
    (dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-1): 2 x T5EncoderBlock(
        (self_attn): MultiHeadAttention(
          (query): Linear(in_features=128, out_features=128, bias=True)
          (key): Linear(in_features=128, out_features=128, bias=True)
          (value): Linear(in_features=128, out_features=128, bias=True)
          (proj): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ffn): FeedForward(
          (net): Sequential(
            (0): Linear(in_features=128, out_features=512, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.1, inplace=False)
            (3): Linear(in_features=512, out_features=128, bias=True)
            (4): Dropout(p=0.1, inplace=False)
          )
        )
        (ln_

### Task 3.3 Train and test translation model


In [32]:
PRINT_EVERY = 50

for epoch in range(5):   # or more epochs later
    for step in range(len(train_pairs)//BATCH_SIZE):
        src, tgt_in, tgt_out = get_safe_batch('train', batch_size=BATCH_SIZE)

        logits = model(src, tgt_in)
        loss = criterion(logits.reshape(-1, vocab_size), tgt_out.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print every few iterations
        if step % PRINT_EVERY == 0:
            print(f"Epoch {epoch} | Step {step}: Loss = {loss.item():.4f}")

print("✓ Training completed successfully!")


Epoch 0 | Step 0: Loss = 89.3607
Epoch 0 | Step 50: Loss = 5.1698
Epoch 0 | Step 100: Loss = 4.1441
Epoch 0 | Step 150: Loss = 3.7162
Epoch 0 | Step 200: Loss = 3.4283
Epoch 0 | Step 250: Loss = 3.2717
Epoch 0 | Step 300: Loss = 3.0852
Epoch 0 | Step 350: Loss = 2.9160
Epoch 0 | Step 400: Loss = 2.8818
Epoch 0 | Step 450: Loss = 2.7830
Epoch 0 | Step 500: Loss = 2.7026
Epoch 0 | Step 550: Loss = 2.7062
Epoch 0 | Step 600: Loss = 2.6180
Epoch 0 | Step 650: Loss = 2.6384
Epoch 0 | Step 700: Loss = 2.5355
Epoch 0 | Step 750: Loss = 2.6297
Epoch 0 | Step 800: Loss = 2.4999
Epoch 0 | Step 850: Loss = 2.5242
Epoch 0 | Step 900: Loss = 2.5133
Epoch 0 | Step 950: Loss = 2.5112
Epoch 0 | Step 1000: Loss = 2.5093
Epoch 0 | Step 1050: Loss = 2.5554
Epoch 0 | Step 1100: Loss = 2.4445
Epoch 0 | Step 1150: Loss = 2.5198
Epoch 0 | Step 1200: Loss = 2.4293
Epoch 0 | Step 1250: Loss = 2.3989
Epoch 0 | Step 1300: Loss = 2.4592
Epoch 0 | Step 1350: Loss = 2.4719
Epoch 0 | Step 1400: Loss = 2.4656
Epoch 0

In [33]:
# Set model to evaluation mode (disables dropout)
model.eval()

def translate_sentence(sentence, model, max_length=100):
    """
    Translates an English sentence to German using the trained T5Model.
    Uses greedy decoding.
    """
    # 1. Prepare Source (English)
    # Encode and add batch dimension [1, seq_len]
    src_ids = encode(sentence)
    src_tensor = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)

    # 2. Encoder Pass
    # We encode the source sequence once
    with torch.no_grad():
        encoder_output = model.encoder(src_tensor)

    # 3. Decoder Loop (Autoregressive)
    # Start with the Start-Of-Sequence token
    decoder_input = torch.tensor([[SOS_IDX]], dtype=torch.long).to(device)

    generated_tokens = []

    for _ in range(max_length):
        with torch.no_grad():
            # Pass current sequence and encoder output to decoder
            # decoder_input shape: [1, current_seq_len]
            decoder_output = model.decoder(decoder_input, encoder_output)

            # Project to vocabulary
            logits = model.lm_head(decoder_output)

            # Get the token with highest probability for the last position
            # logits shape: [1, current_seq_len, vocab_size]
            next_token_logits = logits[:, -1, :]
            next_token_id = torch.argmax(next_token_logits, dim=-1).item()

        # Stop if End-Of-Sequence token is generated
        if next_token_id == EOS_IDX:
            break

        generated_tokens.append(next_token_id)

        # Append prediction to decoder input for next step
        next_token_tensor = torch.tensor([[next_token_id]], dtype=torch.long).to(device)
        decoder_input = torch.cat([decoder_input, next_token_tensor], dim=1)

    # 4. Decode to string
    translated_text = decode(generated_tokens)
    return translated_text

# --- Test Translations ---
print("--- Translation Demo ---")
test_sentences = [
    "Hello",
    "Good morning",
    "I love you",
    "Where is the bathroom?",
    "This is difficult"
]

for s in test_sentences:
    trans = translate_sentence(s, model)
    print(f"EN: {s:25} -> DE: {trans}")

# Try a custom one
custom = "I am hungry"
print(f"\nCustom: {custom} -> {translate_sentence(custom, model)}")

--- Translation Demo ---
EN: Hello                     -> DE: Wie sollten der Bereichen zu erwährleichen Bericht eine lobalen letzten  ollen Bereichen zu erwältig
EN: Good morning              -> DE: Bereich Bericht in der Frage in der Frage der Frage angenen  erreichen Berichterstützung von Fragen 
EN: I love you                -> DE: Ich möchte  möchte   eine  bestehen, wenn  eine  bestehen, wenn  ich möchte   eine   eine  bestehen,
EN: Where is the bathroom?    -> DE: Wir sollten der  eine  sind so hat eine  solchen  in einem  auch eine  solchen  eine  sich nicht nic
EN: This is difficult         -> DE: Diese Statschaft die Statelinie der Statschaftsprechen der Staten der Staten der Staten der Staten d

Custom: I am hungry -> Ich möchte mir gegen Sie eine seine Sitzung nach eine  sich nicht nicht nicht nicht nicht nicht nich


# BONUS (2 points): Solve any task with an LLM

**Goal.**  
Pick **one** of the homework tasks (Task 1, Task 2, or Task 3) and solve it using an **LLM**. Provide the **LLM** with the **task description** and any **starter/prerequisite code** it depends on, ask it to generate a complete **code solution** first, then run that **generated code** here in the code notebook yourself. Finally, document what you did and **compare** the LLM’s result to your own pipeline.

**What to deliver below.**
1) **LLM used** (name + version, e.g., “Llama-3-8B-Instruct”, “GPT-x”, “Claude-x”, “Mistral-x”, etc.).  
2) **Prompt(s)** you used.  
3) **LLM output** — copy and paste the generated code.  
4) **Comparison** to your solution: what matches or differs (quantitative or qualitative).  
5) **Reflection**: what the LLM was **good at** vs **bad at**, what it got **right** vs **wrong**.

> **No code required.** You do **not** need to run, share, or submit any code used for the LLM generation. Provide only the deliverables listed above.
> You may use any LLMs through any interface (API, web UI, local inference).


#LLM used:
1. Google Gemini (“Flash 2.5”)

#Prompt

```
You are helping me implement TASK 3.1 of a homework: Build a complete T5 model for machine translation.
Combine encoder and decoder into a complete T5 model and train it on English-German machine translation.

3.1: Complete T5 Model — Combine encoder and decoder stacks with shared token embeddings and language modeling head
3.2: Translation Dataset Preparation — Prepare English-German translation dataset with proper batching and padding
3.3: Training and Testing — Train model on translation task and evaluate performance

Please do this:
TASK 3.1: Complete T5 Model — Combine encoder and decoder stacks with shared token embeddings and language modeling head.

Constraints:
- Use PyTorch.
- I already implemented T5Encoder and T5Decoder classes that return hidden states of shape:
  encoder(src) -> (B, T_src, d_model)
  decoder(tgt, encoder_output) -> (B, T_tgt, d_model)
- I have PAD_IDX defined globally.
- I want shared token embeddings between encoder and decoder.
- I want a language modeling head that maps decoder hidden states to vocab logits.
- Please implement class T5Model(nn.Module) with __init__ and forward(self, src, tgt) returning logits of shape (B, T_tgt, vocab_size).
- Use weight tying: lm_head weight tied to shared embedding.

Starter signature to fill:
class T5Model(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, block_size, max_len=5000, dropout=0.1):
        super().__init__()
        # TODO

    def forward(self, src, tgt):
        # TODO
        return logits
```

#Google Gemini Output / LLM Output:
```
This is a great task for building a complete T5-style model\! The core idea is to share the input token embeddings between the encoder and the decoder and then use the same weight matrix for the final linear layer (the language modeling head) that maps the decoder's output to the vocabulary logits (a technique called **weight tying**).

Here is the implementation of the `T5Model` class in PyTorch, assuming you have already defined `T5Encoder` and `T5Decoder` that handle the token embedding and positional encoding *internally* based on a shared embedding layer you pass them.

### 💻 T5Model Implementation

```python
import torch
import torch.nn as nn
# Assuming you have these classes defined elsewhere
# from your_module import T5Encoder, T5Decoder

# Mock T5Encoder and T5Decoder for self-contained example
# Replace these with your actual implementations
class T5Encoder(nn.Module):
    def __init__(self, embedding_layer, d_model, n_heads, n_layers, d_ff, dropout):
        super().__init__()
        # In a real implementation, this would include the encoder stack (layers)
        self.embedding = embedding_layer
        self.d_model = d_model
    
    def forward(self, src):
        # Apply embedding and get hidden states
        # The embedding layer is assumed to handle the token-to-vector conversion
        B, T_src = src.shape
        # Mock encoder logic: simple lookup and a dummy projection
        embedded = self.embedding(src)
        # Assuming the embedding layer returns (B, T_src, d_model)
        return embedded

class T5Decoder(nn.Module):
    def __init__(self, embedding_layer, d_model, n_heads, n_layers, d_ff, dropout):
        super().__init__()
        # In a real implementation, this would include the decoder stack (layers)
        self.embedding = embedding_layer
        self.d_model = d_model

    def forward(self, tgt, encoder_output):
        # Apply embedding and get hidden states
        # The embedding layer is assumed to handle the token-to-vector conversion
        B, T_tgt = tgt.shape
        # Mock decoder logic: simple lookup and a dummy projection
        embedded = self.embedding(tgt)
        # Assuming the embedding layer returns (B, T_tgt, d_model)
        # In the real decoder, it would use encoder_output for cross-attention
        return embedded


# --- Your requested implementation starts here ---

class T5Model(nn.Module):
    """
    Complete T5 Model combining encoder and decoder with shared token embeddings
    and a weight-tied language modeling head.
    """
    def __init__(self, vocab_size, d_model, n_heads, n_layers, d_ff, block_size, max_len=5000, dropout=0.1):
        super().__init__()
        
        # 1. Shared Token Embeddings
        # A T5-style model uses the same embedding matrix for both input and output.
        # This layer maps tokens (indices) to vectors (d_model).
        self.shared_embedding = nn.Embedding(vocab_size, d_model)

        # 2. Encoder and Decoder Stacks
        # Pass the shared embedding layer to both the encoder and decoder.
        # Your T5Encoder/T5Decoder classes should handle the rest (Positional Encoding,
        # Transformer Blocks, Layer Norms, etc.) internally.
        self.encoder = T5Encoder(
            embedding_layer=self.shared_embedding,
            d_model=d_model,
            n_heads=n_heads,
            n_layers=n_layers,
            d_ff=d_ff,
            dropout=dropout
        )
        
        self.decoder = T5Decoder(
            embedding_layer=self.shared_embedding,
            d_model=d_model,
            n_heads=n_heads,
            n_layers=n_layers,
            d_ff=d_ff,
            dropout=dropout
        )
        
        # 3. Language Modeling Head (Output Layer)
        # This is a simple linear layer that maps the decoder's hidden states
        # (B, T_tgt, d_model) back to vocabulary logits (B, T_tgt, vocab_size).
        # We initialize it as a standard Linear layer first.
        self.lm_head = nn.Linear(d_model, vocab_size, bias=False)
        
        # 4. Weight Tying
        # The weight matrix of the output linear layer (`lm_head.weight`) is
        # tied to the weight matrix of the shared token embedding layer
        # (`shared_embedding.weight`).
        self.lm_head.weight = self.shared_embedding.weight
        
        # Initialize parameters (often done with a specific scheme like Xavier/Kaiming)
        # For simplicity, we just initialize the lm_head for demonstration.
        self._init_weights()

    def _init_weights(self):
        # A common practice in T5 is to scale the embeddings/outputs
        nn.init.normal_(self.shared_embedding.weight, mean=0.0, std=0.02)
        # No need to initialize lm_head.weight since it's tied to shared_embedding.weight

    def forward(self, src, tgt):
        """
        Processes source and target sequences through the T5 model.

        Args:
            src (Tensor): Source sequence (e.g., English), shape (B, T_src).
            tgt (Tensor): Target sequence (e.g., German, shifted right), shape (B, T_tgt).

        Returns:
            Tensor: Logits over the vocabulary for the next target token, shape (B, T_tgt, vocab_size).
        """
        
        # 1. Encoder Forward Pass
        # Maps the source sequence to contextualized hidden states.
        # encoder_output shape: (B, T_src, d_model)
        encoder_output = self.encoder(src)
        
        # 2. Decoder Forward Pass
        # Processes the target sequence and attends to the encoder output.
        # decoder_output shape: (B, T_tgt, d_model)
        decoder_output = self.decoder(tgt, encoder_output)
        
        # 3. Language Modeling Head
        # Maps the decoder's hidden states to logits over the vocabulary.
        # logits shape: (B, T_tgt, vocab_size)
        logits = self.lm_head(decoder_output)
        
        return logits

# --- End of implementation ---
```

### 💡 Key Design Points Explained

1.  **Shared Token Embedding (`self.shared_embedding`)**:

      * This single `nn.Embedding` layer is created once.
      * It is passed into the constructors of both `self.encoder` and `self.decoder`. This ensures both components use the **identical weight matrix** for looking up token representations, which is a standard T5/Transformer design feature.

2.  **Encoder-Decoder Flow**:

      * The `forward` method first computes `encoder_output = self.encoder(src)`.
      * This output is then passed to the decoder as the context for cross-attention: `decoder_output = self.decoder(tgt, encoder_output)`.

3.  **Language Modeling Head (`self.lm_head`)**:

      * This is the final linear layer that converts the high-dimensional decoder output back to a vector of scores (logits) for every word in the vocabulary.

4.  **Weight Tying (The Crucial Step)**:

      * The line `self.lm_head.weight = self.shared_embedding.weight` is what ties the weights.
      * The weight matrix used for *decoding* (mapping hidden state to vocabulary scores) is set to be the **exact same matrix** used for *encoding* (mapping input tokens to hidden states). This is an effective form of regularization that reduces the number of parameters and often improves performance.

The structure of the T5 model you're building is a classic **Encoder-Decoder Transformer** architecture.

Would you like me to proceed with **TASK 3.2: Translation Dataset Preparation** using PyTorch and the common libraries like HuggingFace's `datasets`?
```

#Comparision:

**What matches (same idea):**

1. Shared embeddings: Both approaches use one embedding matrix for encoder + decoder.
2. LM head: Both map decoder hidden states (B, T_tgt, d_model) to logits (B, T_tgt, vocab_size).
3. Weight tying: Both tie lm_head.weight to the shared embedding weights.
4. Forward flow: Both do encoder(src) → decoder(tgt, encoder_output) → lm_head

**Differences:**
1. Gemini did not use PAD_IDX to build a padding mask. In my pipeline, I have  pass a padding mask into the encoder so attention ignores <pad> tokens.
2. Gemini includes explicit initialization, My pipeline relies on PyTorch default init.
3. Gemini output includes mock encoder/decoder classes (demo code) in the response.
4. Gemini shares embeddings by passing them in, I shared embeddings by replacing the embedding layer after construction.

# Reflection:
**Good at:**
1. Got the overall architecture right.
2. Remembered weight tying and implemented it correctly.
3. Produced readable, structured code.


**Bad at / got wrong:**
1. Assumed a different T5Encoder/T5Decoder API than the notebook’s actual implementation.
2. Included extra mock code and explanations.
3. Did not use PAD_IDX masking, which matters for correct training with padded batches.